# Parametric non-intrusive model order reduction for synthetic data

Here we describe the application of methods sPOD-NN, sPOD-I and POD-NN on a 1D syntheticaly generated data set

In [ ]:
import sys
sys.path.append('../sPOD/lib/')
sys.path.append('../DL-ROM/LIB/')

In [ ]:
from synthetic_sup import synthetic_sup
import numpy as np

## Basis reconstruction for the synthetic data

In this part we : 

<ul>
<li>Generate the 1D synthetic data with $M = 500$ and $N_t = 500$ with $training\_samples$ being the number of different values of parameter considered for basis reconstruction and further network training and $testing\_sample$ being the value of test parameter.</li>
<li>Perform sPOD and POD on the generated data.</li>
<li>Extract the time amplitudes according to Eq.(9) and Eq.(15) from the paper.</li>
</ul>

The inputs here include : 
<ul>
<li>$spod\_iter$ is the number of iterations for the sPOD algorithm.</li>
<li>$nmodes$ are the number of singular values taken into consideration while creating the data.</li>
<li>$plot\_offline\_data$ is the switch variable for the user to plot the results of the basis reconstruction.</li>
</ul>

In [ ]:
df = synthetic_sup(training_samples=[0.1, 0.15, 0.2, 0.25, 0.3], testing_sample=[0.23], 
                   nmodes=8, spod_iter=300, plot_offline_data=False)

In [ ]:
# We collect the time amplitudes, shifts and the parameters for the training as well as the testing data.
TA_TRAIN = df.TA_TRAIN
SHIFTS_TRAIN = df.SHIFTS_TRAIN
PARAMS_TRAIN = df.PARAMS_TRAIN
TA_TEST = df.TA_TEST
SHIFTS_TEST = df.SHIFTS_TEST
PARAMS_TEST = df.PARAMS_TEST
TA_POD_TRAIN = df.TA_POD_TRAIN
TA_POD_TEST = df.TA_POD_TEST

We assemble the $\hat{A}$ matrix according to the Eq.(18) from the paper.

In [ ]:
shifts_train = np.concatenate((np.reshape(SHIFTS_TRAIN[0], newshape=[1, -1]), np.reshape(SHIFTS_TRAIN[1], newshape=[1, -1])), axis=0)
shifts_test = np.concatenate((np.reshape(SHIFTS_TEST[0], newshape=[1, -1]), np.reshape(SHIFTS_TEST[1], newshape=[1, -1])), axis=0)

ta_train = np.concatenate((TA_TRAIN, shifts_train), axis=0)
ta_test = np.concatenate((TA_TEST, shifts_test), axis=0)

In [ ]:
print("Grid, Nx : {}, Nt : {}".format(df.Nx, df.Nt))
print("Number of sPOD frames : {}".format(df.NumFrames))
print("Number of modes per frame : {}".format(df.nmodes))
print("Number of parameter instances : {}".format(int(int(ta_train.shape[1]) / df.Nt)))
print("Size of training matrix : {} x {}".format(int(ta_train.shape[0]), int(ta_train.shape[1])))

## Neural network training

Based on the data which we obtain from the previous step we train our neural network. For the training we first define certain parameters needed for training step. The parameters are mentioned here are:

<ul>
<li>$scaling$ activates the min-max data scaling for efficient training.</li>
<li>$full\_order\_model\_dimension$ is effectively $M$ which is the total number of grid points.</li>
<li>$reduced\_order\_model\_dimension$ is $n_{\mathrm{dof}}$ mentioned in Eq.(21) in the paper.</li>
<li>$totalModes$ is the total number of modes.</li>
<li>$num\_early\_stop$ defines the early stopping criteria for training step.</li>
</ul>

Subsequently the hyperparameters are:
<ul>
<li>$epochs$ sets the total number of epochs for training.</li>
<li>$lr$ sets the learning rate for training.</li>
<li>$loss\_type$ is the type of loss to consider while training options being $L1$ or $MSE$.</li>
<li>$batch\_size$ sets the total number of minibatches for the training data to be broken down into for effective training.</li>
</ul>

In [ ]:
params_sPOD = {
        'scaling': True,  
        'full_order_model_dimension': df.Nx,  
        'reduced_order_model_dimension': ta_train.shape[0], 
        'totalModes': ta_train.shape[0] - df.NumFrames,  
        'num_early_stop': 100000  
    
    }
params_POD = {
        'scaling': True,  
        'full_order_model_dimension': df.Nx, 
        'reduced_order_model_dimension': TA_POD_TRAIN.shape[0], 
        'totalModes': TA_POD_TRAIN.shape[0],  
        'num_early_stop': 500  
    }

In [ ]:
# training the model
from DFNN import run_model 
print("#################################")
print("sPOD-NN")
trained_model_sPOD, _, scaling_sPOD = run_model(ta_train, PARAMS_TRAIN, epochs=150000, lr=0.0025, loss_type='L1', 
                                                logs_folder='./DNN_result/synthetic/training_results_sPOD', 
                                                params=params_sPOD, batch_size=50)
print("#################################\n")
print("#################################")
print("POD-NN")
trained_model_POD, _, scaling_POD = run_model(TA_POD_TRAIN, PARAMS_TRAIN, epochs=150000, lr=0.0025, loss_type='L1',
                                              logs_folder='./DNN_result/synthetic/training_results_POD', 
                                              params=params_POD, batch_size=50)
print("#################################\n")

## Neural network prediction

After the training is finished the best weights are saved for network prediction. Here those weights are loaded and the prediction is performed. The dictionary $test$ is defined here which determines whether to run a multi-query scenario or full prediction scenario. If $test['typeOfTest'] = "query"$ then the multi-query scenario is run for which $test['typeOfTest'] = 200$ sets the time step at which the prediction has to be performed. 

Plotting function is only activated for $test['typeOfTest'] = "full"$ which gives us the full prediction throughout all the time steps.

In [ ]:
test = {
    'typeOfTest': "full",
    'test_sample': 200
}

In [ ]:
# loading the model
from DFNN import scale_params
import torch
import pathlib
import os

log_folder_base_sPOD = 'DNN_result/synthetic/training_results_sPOD/'
log_folder_trained_model_sPOD = sorted(pathlib.Path(log_folder_base_sPOD).glob('*/'), key=os.path.getmtime)[-1]
PATH_sPOD = str(log_folder_trained_model_sPOD) + '/trained_weights/' + 'weights.pt'

log_folder_base_POD = 'DNN_result/synthetic/training_results_POD/'
log_folder_trained_model_POD = sorted(pathlib.Path(log_folder_base_POD).glob('*/'), key=os.path.getmtime)[-1]
PATH_POD = str(log_folder_trained_model_POD) + '/trained_weights/' + 'weights.pt'

PATH_sPOD = 'DNN_result/synthetic/training_results_sPOD/2023_02_10__15-10-53/trained_weights/weights.pt'
PATH_POD = 'DNN_result/synthetic/training_results_POD/2023_02_10__15-20-51/trained_weights/weights.pt'

# Scale the parameters before prediction
if '/trained_weights/weights.pt' in PATH_sPOD: address_sPOD = PATH_sPOD.replace('/trained_weights/weights.pt', '')
scaling_sPOD = np.load(address_sPOD + '/variables/' + 'scaling.npy', allow_pickle=True)

if '/trained_weights/weights.pt' in PATH_POD: address_POD = PATH_POD.replace('/trained_weights/weights.pt', '')
scaling_POD = np.load(address_POD + '/variables/' + 'scaling.npy', allow_pickle=True)

PARAMS_TEST_sPOD = scale_params(PARAMS_TEST, params_sPOD, scaling_sPOD)
PARAMS_TEST_POD = scale_params(PARAMS_TEST, params_POD, scaling_POD)

In [ ]:
if test['typeOfTest'] == "query":
    test_sample = test['test_sample']
    
    ta_test = ta_test[:, test_sample][..., np.newaxis]
    
    df.TA_TEST = df.TA_TEST[:, test_sample][..., np.newaxis]
    df.TA_POD_TEST = df.TA_POD_TEST[:, test_sample][..., np.newaxis]
    
    tmp = []
    for i in range(df.NumFrames):
        tt = []
        for m in range(df.nmodes):
            ampl = df.TA_interp_list[i][m][test_sample, :][np.newaxis, ...]
            tt.append(ampl)
        tmp.append(tt)
    df.TA_interp_list = tmp
    
    df.SHIFTS_TEST[0] = df.SHIFTS_TEST[0][:, test_sample]
    df.SHIFTS_TEST[1] = df.SHIFTS_TEST[1][:, test_sample]
    
    PARAMS_TEST_sPOD = PARAMS_TEST_sPOD[:, test_sample][..., np.newaxis]
    PARAMS_TEST_POD = PARAMS_TEST_POD[:, test_sample][..., np.newaxis]

In [ ]:
# testing the model
from DFNN import test_model 
rel_err_sPOD, results_predicted_sPOD = test_model(ta_test, PARAMS_TEST_sPOD, saved_model=True, 
                                                  PATH_TO_WEIGHTS=PATH_sPOD, params=params_sPOD,
                                                  scaling=scaling_sPOD, batch_size=50) 
rel_err_POD, results_predicted_POD = test_model(TA_POD_TEST, PARAMS_TEST_POD, saved_model=True,
                                               PATH_TO_WEIGHTS=PATH_POD, params=params_POD,
                                               scaling=scaling_POD, batch_size=50)

Once the predictions for the time amplitudes and the shifts had been made we now reconstruct the snapshot according to Eq.(12) and Eq.(20) for POD and sPOD based methods respectively

In [ ]:
TA_sPOD_pred = results_predicted_sPOD[:-2, :]
shifts_sPOD_pred = results_predicted_sPOD[-2:, :]
TA_POD_pred = results_predicted_POD

In [ ]:
errors = df.OnlinePredictionAnalysis(TA_sPOD_pred, shifts_sPOD_pred, TA_POD_pred, plot_online=False, test_type=test)